In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

In [2]:
dfTrain = pd.read_csv('../../../DataSet/trip_train.csv', 
                      parse_dates=['start_date', 'end_date'], 
                      infer_datetime_format=True)

In [3]:
'''Funcion bastante garompa que devuelve los numeros primos hasta max_number'''
def primeNumbersUntil(max_number):
    primes = [2]
    for num in range(3,max_number,2):
        if all(num%i!=0 for i in range(2,int(math.sqrt(num))+1)):
           primes.append(num)
    return primes

In [4]:
# Convierto a los SUSCRIBER en un 0
# Convierto a los CUSTOMER en un 1
subscriptionTypes = dfTrain.subscription_type.unique()
print subscriptionTypes

# Reemplazo por 0 y 1
dfTrain.subscription_type = dfTrain.subscription_type.astype('category', categories=subscriptionTypes).cat.codes

['Subscriber' 'Customer']


In [5]:
# GENERO TARGET, TRAIN
target = dfTrain.duration

trainDateData = {'start_month':dfTrain.start_date.dt.month,
                 'start_dayOfYear':dfTrain.start_date.dt.dayofyear,
                 'start_dayOfWeek':dfTrain.start_date.dt.dayofweek, 
                 'start_hourOfDay':dfTrain.start_date.dt.hour}

# Agrego columnas con la informacion de fechas a los dataframes
dfTrain = dfTrain[['start_station_id', 'subscription_type']].join(pd.DataFrame(trainDateData), how='outer')

In [6]:
kValues = primeNumbersUntil(100)
cvScores = []
for k in kValues:
    knn = KNeighborsRegressor(n_neighbors=k, n_jobs=-1)
    scores = cross_val_score(knn, dfTrain, target, scoring='neg_mean_squared_error', n_jobs=-1)
    cvScores.append(scores.mean())
kValues[cvScores.index(min(cvScores))]

3